## [Running Flask and FastAPI on Google Colab](https://medium.datadriveninvestor.com/flask-on-colab-825d2099d9d8)

In [1]:
!pip install fastapi nest-asyncio pyngrok uvicorn

     |████████████████████████████████| 51kB 2.2MB/s 
     |████████████████████████████████| 747kB 7.3MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 
     |████████████████████████████████| 10.1MB 12.2MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 92kB 9.1MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-cp37-none-any.whl size=19246 sha256=4af281d4af546b3713f56a91856abcf24e0ab6d6e31902d0eabd15be4893ef02
  Stored in directory: /root/.cache/pip/wheels/0c/13/64/5ebbcc22eaf53fdf5766b397c1fb17c83f5775fdccf0ea1b88
Successfully built pyngrok
  Found existing installation: click 8.0.0
    Uninstalling click-8.0.0:
      Successfully uninstalled click-8.0.0


In [2]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn

app = FastAPI()

@app.get('/index')
async def home():
  return "Hello World"

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
print('Public docs URL:', ngrok_tunnel.public_url + "/docs")
nest_asyncio.apply()
uvicorn.run(app, port=8000)


INFO:     Started server process [59]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: http://13a7db702ebc.ngrok.io
Public docs URL: http://13a7db702ebc.ngrok.io/docs


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [59]


## [Deploying Transformer Models](https://chatbotslife.com/deploying-transformer-models-1350876016f)


In [3]:
%%bash
pip install -qq transformers
#pip install torch torchvision
pip install "fugashi[unidic-lite]" 
pip install ipadic

  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-cp37-none-any.whl size=47658825 sha256=157022620f15e35c3d06b6734a8c565734002f50fea16b49ca9e3c2d727ed352
  Stored in directory: /root/.cache/pip/wheels/20/48/8d/b66d8361a27f58f41ec86640e4fd2640de0403a6367511eab7
Successfully built unidic-lite
  Created wheel for ipadic: filename=ipadic-1.0.0-cp37-none-any.whl size=13556725 sha256=0f6ed3d878443e0a3a0e7c27a0112e3f2f2915ee74ac14fe3971e74fe2658501
  Stored in directory: /root/.cache/pip/wheels/ff/00/d1/0c094a0ce58a77199a0c5801f0ecf510c80f0ecbec27f07d2c
Successfully built ipadic


In [33]:
import torch
from transformers import (
    pipeline,
    AutoModelForMaskedLM,
    AutoTokenizer
)

class NLP:
    def __init__(self):
        self.gen_tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
        self.gen_model = AutoModelForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
         
    def generate(self, prompt="The epistemelogical limit"):
        #inputs = self.gen_tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)
        #with torch.no_grad():
        #    summary_ids = self.gen_model.generate(inputs) #, max_length=512, min_length=5, length_penalty=5., num_beams=2)
        #    summary = self.gen_tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        #    return summary

        # 入力テキストのエンコード
        input_ids = self.gen_tokenizer.encode(f'吾輩は{self.gen_tokenizer.mask_token}である。名前はまだ無い。', return_tensors='pt', max_length=512, truncation=True)
        #input_ids = self.gen_tokenizer.encode(prompt, return_tensors='pt', max_length=512, truncation=True)
        print('input_ids:', self.gen_tokenizer.convert_ids_to_tokens(input_ids[0].tolist()))
        
        # マスクインデックスの取得
        masked_index = torch.where(input_ids == self.gen_tokenizer.mask_token_id)[1].tolist()[0]
        print('masked_index:', masked_index)
        
        # マスクトークンの予測
        result = self.gen_model(input_ids)
        pred_ids = result[0][:, masked_index].topk(5).indices.tolist()[0]

        output = []
        for pred_id in pred_ids:
            output_ids = input_ids.tolist()[0]
            output_ids[masked_index] = pred_id
            #print(self.gen_tokenizer.decode(output_ids))
            #print(self.gen_tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True))
            output.append(self.gen_tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True))
        return output


In [35]:
# from nlp import NLP
nlp = NLP()
#print(nlp.sentiments("うほほーい、大好き♡"))
for s in nlp.generate():
    print(s)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


input_ids: ['[CLS]', '吾', '##輩', 'は', '[MASK]', 'で', 'ある', '。', '名前', 'は', 'まだ', '無い', '。', '[SEP]']
masked_index: 4
吾輩 は 猫 で ある 。 名前 は まだ 無い 。
吾輩 は 犬 で ある 。 名前 は まだ 無い 。
吾輩 は 人間 で ある 。 名前 は まだ 無い 。
吾輩 は 狼 で ある 。 名前 は まだ 無い 。
吾輩 は 私 で ある 。 名前 は まだ 無い 。


In [36]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
#from app.nlp import NLP
import nest_asyncio
from pyngrok import ngrok
import uvicorn

class Message(BaseModel):
    input: str
    output: str = None

app = FastAPI()
nlp = NLP()

origins = [
    "http://localhost",
    "http://localhost:3000",
    "http://127.0.0.1:3000"
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["POST"],
    allow_headers=["*"],
)

@app.post("/generative/")
async def  generate(message: Message):
    message.output  = nlp.generate(prompt=message.input)
    return {"output" : message.output}

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
print('Public docs URL:', ngrok_tunnel.public_url + "/docs")
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Public URL: http://5d97a0bc7adc.ngrok.io
Public docs URL: http://5d97a0bc7adc.ngrok.io/docs


INFO:     Started server process [59]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     124.32.186.1:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     124.32.186.1:0 - "GET /openapi.json HTTP/1.1" 200 OK
input_ids: ['[CLS]', '吾', '##輩', 'は', '[MASK]', 'で', 'ある', '。', '名前', 'は', 'まだ', '無い', '。', '[SEP]']
masked_index: 4
INFO:     124.32.186.1:0 - "POST /generative/ HTTP/1.1" 200 OK
INFO:     124.32.186.1:0 - "POST /generative/ HTTP/1.1" 422 Unprocessable Entity
input_ids: ['[CLS]', '吾', '##輩', 'は', '[MASK]', 'で', 'ある', '。', '名前', 'は', 'まだ', '無い', '。', '[SEP]']
masked_index: 4
INFO:     124.32.186.1:0 - "POST /generative/ HTTP/1.1" 200 OK
input_ids: ['[CLS]', '吾', '##輩', 'は', '[MASK]', 'で', 'ある', '。', '名前', 'は', 'まだ', '無い', '。', '[SEP]']
masked_index: 4
INFO:     124.32.186.1:0 - "POST /generative/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [59]


```
% curl -X 'POST' \
  'http://5d97a0bc7adc.ngrok.io/generative/' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "input": "",
  "output": ""
}'

{"output":["吾輩 は 猫 で ある 。 名前 は まだ 無い 。","吾輩 は 犬 で ある 。 名前 は まだ 無い 。","吾輩 は 人間 で ある 。 名前 は まだ 無い 。","吾輩 は 狼 で ある 。 名前 は まだ 無い 。","吾輩 は 私 で ある 。 名前 は まだ 無い 。"]}
```